In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob

import joblib

from utils import get_features, text_embedding

from keras.models import load_model
from sentence_transformers import SentenceTransformer

c:\Users\82105\anaconda3\envs\cpuvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
current_path=os.getcwd().replace("\\", "/").replace("c:", "C:")
data_path=current_path+'/data/'
model_path=current_path+"/model/"

In [3]:
model = load_model(model_path+'my_model.h5')
encoder = joblib.load(model_path+'encoder.pkl')
scaler = joblib.load(model_path+'scaler.pkl')
with open(model_path+'text_model_name.txt', 'r') as file:
    text_model_name = file.read().strip()

In [5]:
txt_data=pd.read_csv(data_path+"label/original.csv")

In [46]:
txt_data[txt_data["final_label"]=="happiness"].sample(3)

,wav_id,final_label,sentence
11367,5f5cc14d54b23616212847f9,happiness,친구들하곤 벌써 약속을 다 잡아놨지.
42615,5f9921e3f8fac448cc0a858f,happiness,좀 건강은 아주 좋아.
14627,5f61ec172e23c7161accd3ef,happiness,나 축하해줘! 기분 좋은 일이 있어.


In [47]:
file_name='5f61ec172e23c7161accd3ef'
new_audio_path = data_path+"/wavFiles/"+file_name+'.wav'
new_audio_features = get_features(new_audio_path)
new_audio_features_df = pd.DataFrame([new_audio_features])

new_audio_features_df['wav_id']=file_name
new_audio_features_df['final_label']=txt_data[txt_data['wav_id']==file_name]['final_label'].values[0].lower()
new_audio_features_df['sentence']=txt_data[txt_data['wav_id']==file_name]['sentence'].values[0]

In [48]:
txt_embed = text_embedding(model_name = text_model_name)
new_audio_features_df = txt_embed.transform(new_audio_features_df)

# 특징 표준화
new_audio_features_scaled = scaler.transform(new_audio_features_df)

# 모델에 맞게 차원 확장
new_audio_features_scaled = np.expand_dims(new_audio_features_scaled, axis=2)

# 예측 수행
prediction = model.predict(new_audio_features_scaled)
predicted_label = np.argmax(prediction, axis=1)

1/1 [==============================] - 0s 18ms/step


In [49]:
predicted_label

array([3], dtype=int64)

In [50]:
num_categories = encoder.categories_[0].size  # OneHotEncoder에 의해 인코딩된 카테고리 수

# 모델의 예측 결과를 원-핫 인코딩 형식으로 변환
predicted_one_hot = np.zeros((predicted_label.size, num_categories))
predicted_one_hot[np.arange(predicted_label.size), predicted_label] = 1

# 원-핫 인코딩된 예측 결과를 실제 레이블로 변환
actual_label = encoder.inverse_transform(predicted_one_hot)


In [51]:
actual_label[0][0]

'happiness'

In [52]:
# 출력 형식 설정
np.set_printoptions(precision=6, suppress=True)

In [53]:
prediction*100

array([[ 0.189899,  0.000172,  0.      , 99.55067 ,  0.253809,  0.005461]],
      dtype=float32)

In [54]:
prediction

array([[0.001899, 0.000002, 0.      , 0.995507, 0.002538, 0.000055]],
      dtype=float32)